# Questions
- What are git remotes, how do they work, how do they edit them, how do I fix mine
- How do I push to my main branch of git
- Follow up on CAPS question on tokenization section

# Extract raw data
### Expected Format: 
<font color='red'>[</font>{<font color='green'>'text':</font> 'All services should have a dedicated channel for reporting fraud. We see and hear about a lot of fraud on small platforms, especially dating sights and job boards. ',
  <font color='green'>'label':</font> 0,
  <font color='green'>'summary':</font> 'Every service should have a dedicated reporting channel for fraud, because there is lots of fraud on a range of platforms. '}, <font color='red'>...]</font>

In [1]:
# MS: create training set, evaluation set and validation set
import sys
import pandas as pd
import csv

dataset_name = "/home/azureuser/cloudfiles/code/Users/Omololu.Makinde/Llama_tutorial/data/consultation2.csv"
dataset = []

with open(dataset_name, encoding='utf-8-sig') as FID:
    csvReader = csv.DictReader(FID, delimiter="\t")
    for key, row in enumerate(csvReader): 
        # print(key, row)
        dataset.append({
            "label" : row['Topic'].strip().lower().replace('\n', ''),
            "text" : row['Full summary of comment'],
            "summary" : row['One-line summary']
        })

df_dataset = pd.DataFrame(dataset)
df_dataset.drop(columns =['summary'], inplace=True)  # don't need this for now
display(df_dataset)

# Data Check

In [2]:
df_dataset_count = df_dataset['label'].value_counts()
df_dataset['label'].value_counts()

label
approach to the codes                                                               468
automated content moderation (user to user)                                         270
governance and accountability                                                       228
user reporting and complaints (u2u and search)                                      140
content moderation (user to user)                                                   128
user access to services (u2u)                                                       111
enhanced user control (u2u)                                                         101
default settings and user support (u2u)                                              89
cumulative assessment                                                                76
terms of service and publicly available statements                                   60
recommender system testing (u2u)                                                     59
content moderation (search

In [3]:
keep_rows = list(df_dataset_count[df_dataset_count > 10].index)
df_dataset = df_dataset[df_dataset['label'].isin(keep_rows)]
display(df_dataset['label'].value_counts())

label
approach to the codes                                 468
automated content moderation (user to user)           270
governance and accountability                         228
user reporting and complaints (u2u and search)        140
content moderation (user to user)                     128
user access to services (u2u)                         111
enhanced user control (u2u)                           101
default settings and user support (u2u)                89
cumulative assessment                                  76
terms of service and publicly available statements     60
recommender system testing (u2u)                       59
content moderation (search)                            52
service design and user support (search)               30
automated content moderation (search)                  25
statutory tests                                        22
Name: count, dtype: int64

# Convert labels to binary classifier format
- 'governance and accountability' = 1, everything else = 0

Note data set heavily imbalanced so accuracy would be a poor metric of evaluating our model (in its current state)

In [4]:
import numpy as np

# add in our classifier labels
topic = 'governance and accountability'
df_dataset['label'] = np.where(np.array(df_dataset['label']) == topic, 1, 0)
df_dataset['label'].value_counts(normalize=False)

label
0    1631
1     228
Name: count, dtype: int64

# Extract Evaluation Set

In [25]:
# train and test data
class_len = len(df_dataset[df_dataset['label'] == 1])  # find how many values we can take and still have a balanced class
class_0_data = df_dataset[df_dataset.label.eq(0)].sample(class_len) 
class_1_data = df_dataset[df_dataset.label.eq(1)].sample(class_len)
train_test_data = pd.concat([class_0_data, class_1_data])  # 50/50 class split
display(train_test_data)

# evaluation data
eval_data = df_dataset.drop(train_test_data.index)  # put the rest into an evaluation set we can play with
eval_data

,label,text
1882,0,Disadvantages of each signal ● Blocking by Dev...
677,0,The vast majority of file-storage and file-sha...
1571,0,Same answer as Question 13
1329,0,creation of dedicated reporting mechanisms and...
1682,0,5Rights state they are concerned that the draf...
...,...,...
1257,1,there may be some potential downsides to this:...
1268,1,Tracking new illegal content – Any service sho...
895,1,Response: To prevent stifling innovation and e...
470,1,Executive remuneration is often tied to delive...


,label,text
0,0,All services should have a dedicated channel f...
7,0,"When prioritising what content to review, rega..."
8,0,"(Disagree). As noted before, ""there should be ..."
12,0,The RSPCA agrees that the most onerous measure...
13,0,The RSPCA agrees that the most onerous measure...
...,...,...
1878,0,Risk of misuse and exploitation of verificatio...
1879,0,"It should be noted that Snap, as a U.S. compan..."
1881,0,It is Snap’s practice to generally apply accou...
1884,0,Notwithstanding an appeal being successful and...


# Transform data to usable version for huggingface model

### Using distilbert base uncased model
- BERT = is an LLM (large language model)
- distil = smaller model with most of the same power as a normal bert model
- base = not much tweaking or tuning done yet
- uncased = captilised letters make no difference i.e. england = EnglaND (Note: probably want to still consider this in case we change to an cased model)

### Tokenizing our data using our base model
- Tokenizing is breaking our data up into smaller parts for our model to read (note that tokens are always just words but can be special chars or subwords)
- Embedding (not used for this model) take individual token and turning it into a more computer friendly format through transforming it into a hidden multi-dimensional-numeric format e.g. cat -> (12, 45))
- Work from huggingface dataset 
- Only want to tokenize our text
- Pad using special characters (adds length up to the min tensor value) the values if necessary
- Truncate (shortens to the max tensor value) the values if necessary (up to max length)

In [29]:
from datasets import Dataset
from transformers import AutoTokenizer

huggingface_data = Dataset.from_pandas(train_test_data, preserve_index=False)  # don't include pandas index

model_name = "distilbert/distilbert-base-uncased"  # This is our base model

tokenizer = AutoTokenizer.from_pretrained(model_name)
def proc_data(data):
    return tokenizer(data['text'], max_length=512, padding=True, truncation=True)

tokenized_data = huggingface_data.map(proc_data, batched=True)  # advantage of ".map" is we can parallel process data in batches (i think)
print(tokenized_data['text'] == huggingface_data['text'])  # SHOULDN'T THIS NOT BE TRUE??? ESPECIALLY IF I CHANGE MAX_LENGTH TO BE 1 OR SOMETHING

split_tokenized_hugginface_data = tokenized_data.train_test_split(test_size=0.15)  # 85/15 train/test split

train, test = split_tokenized_hugginface_data
print(split_tokenized_hugginface_data)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 456/456 [00:00<00:00, 1941.65 examples/s]

True
DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 387
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 69
    })
})


# Train model
- 1 = "Positive" -> the model is predicting that the input CAN be caterogirsed by our input topic
- 0 = "Negative" -> the model is predicting that the input can NOT be caterogirsed by our input topic

In [30]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

evaluation_metrics = ["accuracy", "f1", "precision", "recall"]

2024-05-10 11:37:58.567870: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 11:38:00.530128: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
